In [7]:
!pip install pyrosettacolabsetup

ERROR: Could not find a version that satisfies the requirement pyrosetta (from versions: none)
ERROR: No matching distribution found for pyrosetta
You should consider upgrading via the 'C:\Users\Santiago\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()

ModuleNotFoundError: No module named 'pyrosetta'

In [3]:
from pyrosetta import *
init()

ModuleNotFoundError: No module named 'pyrosetta'

In [ ]:
import pandas as pd

In [ ]:
from pyrosetta.teaching import *

### BEGIN SOLUTION
sfxn = get_score_function(True)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard res

In [ ]:
# Primero paso: identificar la diferencia con la secuencia original

In [ ]:
# Se importa el dataset con el que se evalua la competencia.
test = pd.read_csv('test.csv')

In [ ]:
copyTest = test.copy()

In [ ]:
# Se usa la secuencia original (se importa la secuencia original)
excusa = pyrosetta.pose_from_pdb('excusa.pdb')

core.import_pose.import_pose: File 'excusa.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 1 in file excusa.pdb. Best match rsd_type:  VAL:NtermProteinFull
core.conformation.Conformation: Found disulfide between residues 47 177
core.conformation.Conformation: current variant for 47 CYS
core.conformation.Conformation: current variant for 177 CYS
core.conformation.Conformation: current variant for 47 CYD
core.conformation.Conformation: current variant for 177 CYD
core.conformation.Conformation: Found disulfide between residues 52 188
core.conformation.Conformation: current variant for 52 CYS
core.conformation.Conformation: current variant for 188 CYS
core.conformation.Conformation: current variant for 52 CYD
core.conformation.Conformation: current variant for 188 CYD


In [ ]:
# Pero tenemos que obtener la secuencia como tal, lo anterior es un archivo
secuenciaOriginal = excusa.sequence()

In [ ]:
copyTest.head()

,seq_id,protein_sequence,pH,data_source,Change
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17E
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17K
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,17X
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,18C
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,18F


In [ ]:
# Lo que realiza la función es identificar la diferencia entre la secuencia original y las demás secuencias 
def identificarCambio(secuenciaVer):
  if secuenciaVer == secuenciaOriginal:
    return 'Original'
  # Si su longitud es diferente significa que se ha eliminado un elemento, toca identificar en donde se realizó 
  if len(secuenciaVer) != len(secuenciaOriginal):
    for i in range(len(secuenciaVer)):
      if secuenciaOriginal[i] != secuenciaVer[i]:
        return str(i+1)+'X'
    return '221X'
  # O directamente se reemplazó un elemento de la secuencia
  for i in range(len(secuenciaOriginal)):
    if secuenciaOriginal[i] != secuenciaVer[i]:
      return str(i+1)+secuenciaVer[i]

In [ ]:
copyTest['Change'] = [identificarCambio(elemento) for elemento in copyTest['protein_sequence']]

In [ ]:
# Segundo paso: EL score de la nueva secuencia

In [ ]:
def producirScore(cambio):
  if cambio == 'Original':
    # sfxn es la función que nos otorga el score
    return sfxn(excusa)
  secuenciaLocal = pyrosetta.pose_from_pdb('excusa.pdb')
  # .delete_region nos permite eliminar cierto elemento de la secuencia, en el caso de que la secuencia haya tenido una eliminación de una de sus partes
  if cambio[-1] == 'X':
    pyrosetta.rosetta.protocols.grafting.delete_region(secuenciaLocal,int(cambio[:-1]),int(cambio[:-1]))
    #return float('nan')
  # Directamente se modifica una parte de la secuencia, pero es una modificación directa (no se elimina)
  else:
    pyrosetta.toolbox.mutants.mutate_residue(secuenciaLocal, int(cambio[:-1]), cambio[-1])
  return sfxn(secuenciaLocal)

In [ ]:
# Se producen los scores
copyTest['Score'] = [producirScore(elemento) for elemento in copyTest['Change']]

Se han truncado las últimas 5000 líneas del flujo de salida.
core.conformation.Conformation: current variant for 177 CYS
core.conformation.Conformation: current variant for 47 CYD
core.conformation.Conformation: current variant for 177 CYD
core.conformation.Conformation: Found disulfide between residues 52 188
core.conformation.Conformation: current variant for 52 CYS
core.conformation.Conformation: current variant for 188 CYS
core.conformation.Conformation: current variant for 52 CYD
core.conformation.Conformation: current variant for 188 CYD
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
core.import_pose.import_pose: File 'excusa.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 1 in file 

In [ ]:
copy2 = copyTest.copy()

In [ ]:
# Para el orden de los scores se debe de multiplicar por -1
copy2['Score'] = - copy2['Score']

In [ ]:
copy2.head()

,seq_id,protein_sequence,pH,data_source,Change,Score
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17E,290.888728
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17K,291.078975
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,17X,249.073499
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,18C,298.155866
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,18F,-201.314127


In [ ]:
#copy2.loc[copy2['Score'].isna(), 'Score'] = copy2.loc[~copy2['Score'].isna()].quantile(q=0.25)['Score']

In [ ]:
# Se hace el ranking de los scores
from scipy.stats import rankdata
copy2['tm'] = rankdata(copy2['Score'])

In [ ]:
result = copy2.copy()

In [ ]:
result.drop(columns=['protein_sequence','pH','data_source','Change','Score'],inplace=True)

In [ ]:
result.head()

,seq_id,tm
0,31390,2355.0
1,31391,2360.0
2,31392,623.0
3,31393,2397.0
4,31394,142.0


In [ ]:
# Se exporta el csv que nos piden en la competencia
result.to_csv('result.csv',index=False)